In [1]:
from ethpandaops_python.preprocessor import Preprocessor
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn


nest_asyncio.apply()
pn.extension("plotly", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
# labeled blobs - https://dune.com/queries/3521610
arbitrum: str = "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc"
linea: str = "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e"
base: str = "0x5050F69a9786F081509234F1a7F4684b5E5b76C9"
starknet: str = "0x2C169DFe5fBbA12957Bdd0Ba47d9CEDbFE260CA7"

num_days: int = 7

In [3]:
preprocessor: Preprocessor = Preprocessor(
    blob_producer=base,
    period=num_days,
    network="mainnet",  # mainnet
)

In [4]:
slot_count_breakdown_df: pl.DataFrame = preprocessor.create_slot_count_breakdown_df()

slot_inclusion_df: pl.DataFrame = (
    preprocessor.create_slot_inclusion_df()
    # there is an outlier in the data that has an unusually high slot count, I think it might be an error with the xatu mempool data.
    # additionally there is currently a single tx with 5 blobs and 1 with 7. Since it's a single row, I think it's easier to just discard and ignore for now
    .filter(pl.col("slot_inclusion_rate") < 250).filter(
        pl.col("slot_inclusion_rate_50_blob_avg") < 40
    )
)

slot_gas_bidding_df: pl.DataFrame = preprocessor.create_slot_gas_bidding_df().filter(
    pl.col("slot_inclusion_rate") < 250
)

slot_gas_groupby_df = preprocessor.create_bid_premium_df().filter(
    pl.col("slot_inclusion_rate") < 250
)

In [5]:
# Histogram count breakdown
slot_chance_bar_chart = slot_count_breakdown_df.plot.barh(
    ylabel="number of txs",
    title="Slot Inclusion Breakdown",
    stacked=True,
    width=1000,
    height=400,
)

### Slot Inclusion Time Charts

In [6]:
slot_inclusion_df.columns

['versioned_hash',
 'nonce',
 'event_date_time_min',
 'event_date_time_max',
 'blob_hashes_length',
 'blob_sidecars_size',
 'fill_percentage',
 'blob_gas',
 'blob_gas_fee_cap',
 'gas_price',
 'gas_tip_cap',
 'gas_fee_cap',
 'hash',
 'from',
 'to',
 'submission_count',
 'slot',
 'slot_time',
 'block_root',
 'kzg_commitment',
 'meta_network_name',
 'blob_size',
 'blob_empty_size',
 'beacon_inclusion_time',
 'slot_inclusion_rate',
 'slot_inclusion_rate_50_blob_avg',
 '2_slot_target_inclusion_rate']

In [7]:
slot_inclusion_line_chart = slot_inclusion_df.select(
    "slot_time",
    "slot_inclusion_rate",
    "slot_inclusion_rate_50_blob_avg",
    "2_slot_target_inclusion_rate",
    "submission_count",
).plot.line(
    x="slot_time",
    y=[
        "slot_inclusion_rate",
        "slot_inclusion_rate_50_blob_avg",
        "2_slot_target_inclusion_rate",
    ],
    color=["blue", "red", "black"],
    ylabel="Beacon Block Inclusion (block)",
    xlabel="Slot Date Time",
    title="Historical Slot Inclusion",
    width=1000,
    height=400,
    shared_axes=True,
)

### Priority Fee and Faster Slot Inclusion

In [8]:
# priority fee scatter plot
priority_fee_premium_chart = slot_gas_bidding_df.sort(
    by="slot_inclusion_rate"
).plot.scatter(
    x="slot_inclusion_rate",
    y="priority_fee_bid_percent_premium",
    width=1000,
    height=400,
)

line_chart_bid_premium = slot_gas_groupby_df.rename(
    {
        "priority_fee_bid_percent_premium": "priority fee bid premium (%)",
        "effective_gas_price_gwei": "block gas price (gwei)",
    }
).plot.line(
    x="slot_inclusion_rate",
    y=["priority fee bid premium (%)"],
    color=["g"],
    ylabel="bid bid premium (%, gwei)",
    xlabel="slot inclusion rate",
    title="priority fee bid premium effect on slot inclusion rate",
)

# ! 4/14/24 Not being used
# line_chart_gas= (
#     slot_gas_bidding_df.group_by("slot_inclusion_rate")
#     .agg(
#         pl.col("priority_fee_bid_percent_premium").median(),
#         pl.col("effective_gas_price_gwei").mean(),
#     )
#     .sort(by="slot_inclusion_rate")
#     .drop_nulls()
#     .rename(
#         {
#             "priority_fee_bid_percent_premium": "priority fee bid premium (%)",
#             "effective_gas_price_gwei": "block gas price (gwei)",
#         }
#     )
#     .plot.line(
#         x="slot_inclusion_rate",
#         y=["block gas price (gwei)"],
#         color=["r"],
#         ylabel="bid premium (%, gwei)",
#         title="gas bid premium probability distributions",
#     )
# )

In [9]:
(priority_fee_premium_chart * line_chart_bid_premium)

:Overlay
   .Scatter.I :Scatter   [slot_inclusion_rate]   (priority_fee_bid_percent_premium)
   .Curve.I   :Curve   [slot_inclusion_rate]   (priority fee bid premium (%))

### Assemble Dashboard

In [10]:
dash = pn.Column(
    pn.pane.Markdown(
        """
        # EIP-4844 Slot Inclusion Dashboard

        ## About
        This dashboard shows detailed analytics for blobs and how fast they are included into the next slot as well as the efficiency of using EIP-1559 priority fees
        as a bidding mechanism for faster slot inclusion. As of April 11, 2024, the data is currently restricted to Base posting data over a 5 day period with 
        plans to open up to other rollups soon. This dashboard is made using [Xatu Data](https://github.com/ethpandaops/xatu-data?tab=readme-ov-file) for EL mempool and Beacon chain data and [Hypersync](https://github.com/enviodev/hypersync-client-python) 
        for transaction gas data for the [EIP-4844 data challenge](https://esp.ethereum.foundation/data-challenge-4844).

        ## Calculating Blob Inclusion in the Beacon Chain
        When a transaction is resubmitted with updated gas parameters, the transaction hash changes. For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. 
        On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains 
        the same blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
        
        We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
        In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
            """
    ),
    pn.pane.Markdown(
        """
        ## Slot Inclusion Rates
        **Slot Inclusion Rate** - The slot inclusion rate indicates the number of slots required for a blob to be included in the beacon chain, with a higher rate signifying a longer inclusion time. 
        The accompanying time-series chart tracks this metric from initial mempool appearance to final beacon block inclusion. A 50 blob slot inclusion average is taken to smooth out the performance. 
        The target slot inclusion rate is 2. 
            """
    ),
    pn.Row(slot_inclusion_line_chart.opts(axiswise=True)),
    pn.pane.Markdown(
        """
        ## EIP-1559 Priority Fee Premium Correlation with Slot Rates
        The scatterplot illustrates the relationship between the EIP-1559 priority fee bid premiums and slot inclusion rates. 
        A higher priority fee bid premium tends to coincide with longer slot inclusion times. This unexpected twist underscores the value of efficient slot utilization.
        The data indicates a trend where higher bid premiums are associated with longer slot inclusion times, 
        suggesting that as the time for a blob to be included in the beacon chain increases, so does the priority fee bid premium.
        This behavior comes from the fact that if a blob sits in the mempool for too long, then it is resubmitted with a higher priority fee. In the case of Base, if the blob doesn't get confirmed within 2 minutes, 
        then it will be resubmitted with doubled fee parameters.
            """
    ),
    pn.Row((priority_fee_premium_chart * line_chart_bid_premium)),
    pn.Column(
        pn.pane.Markdown(
            """
            ## Slot Inclusion Breakdown
            This bar chart represents the number of transactions that are being included within 1 slot, 2 slots, and 3+ slots. 
            * 1 slot = fastest possible inclusion time
            * 2 slots = good inclusion time
            * 3+ slots = slow inclusion time
            """
        ),
        slot_chance_bar_chart.opts(axiswise=True),
    ),
    pn.pane.Markdown(
        """
        ## Summary of Results
        - Blobs have a highly variable slot inclusion rate, with a nontrivial amount of blobs taking 3+ slots to be included.   
        - Higher EIP-1559 priority fees correlate with longer slot inclusion rates.
        - Uncertainty around block gas and priority fee bidding premiums increase with longer slot inclusion rates.
        
"""
    ),
)

In [11]:
# dash.show()
dash.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'da089eb6-d3c4-438a-9d34-7b35a17dd2f6': {'version…